In [2]:
import os
import getpass

usuario = getpass.getuser()

# os.environ['GDAL_DATA'] = f'/opt/anaconda3/envs/geoenv/share/gdal'
# os.environ['PROJ_LIB'] = f'/opt/anaconda3/envs/geoenv/share'

# en windows
os.environ['GDAL_DATA'] = f'C:\\Users\\{usuario}\\anaconda3\\envs\\advprog12\\Library\\share\\gdal'
os.environ['PROJ_LIB'] = f'C:\\Users\\{usuario}\\anaconda3\\envs\\advprog12\\Library\\share\\proj'

In [3]:
import sys

#sys.argv[1] = "C:\\Users\\angel\\Documents\\Programacion_avanzada\\practicas\\Geodatos\\" #Ángel
sys.argv[1] = "E:\\advprog\\p1\\Geodatos\\" #Edu

euc_points = sys.argv[1] + "eucaliptos_españa_puntos\\eucalyptus_points.csv"
euc_mfe = sys.argv[1] + "Eucaliptales_Cornisa_Cantabrica\\Eucaliptales_Cornisa_Cantabrica.shp"


In [4]:
import rasterio
import glob
import numpy as np
from rasterio.merge import merge
from rasterio.enums import Resampling

La siguiente celda genera una lista con las rutas de nuestras 4 imagenes que queremos transformar a multibanda.

In [5]:
#ruta1 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta2 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta3 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta4 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
ruta1 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\\GRANULE\\L2A_T30TUN_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta2 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\\GRANULE\\L2A_T30TUP_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta3 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\\GRANULE\\L2A_T30TVN_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta4 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\\GRANULE\\L2A_T30TVP_A002395_20250219T111258\\IMG_DATA\\R20m"
rutas = []
rutas.append(ruta1)
rutas.append(ruta2)
rutas.append(ruta3)
rutas.append(ruta4)
rutas

['S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\\GRANULE\\L2A_T30TUN_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\\GRANULE\\L2A_T30TUP_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\\GRANULE\\L2A_T30TVN_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\\GRANULE\\L2A_T30TVP_A002395_20250219T111258\\IMG_DATA\\R20m']

El siguiente genera las 4 imagenes en las distintas rutas.

In [5]:
salidas_raster = []
for i, ruta in enumerate(rutas):
    raster_files = sorted(glob.glob(sys.argv[1] + "imagenes/" + ruta + "/*.jp2"))
    
    if not raster_files:
        print(f"No se encontraron archivos en {ruta}")
        continue
    
    bands = []
    with rasterio.open(raster_files[0]) as src:
        meta = src.meta.copy()
        meta.update({"count": len(raster_files), "driver": "GTiff"})
    
    for file in raster_files:
        with rasterio.open(file) as src:
            bands.append(src.read(1))
    
    stacked_array = np.stack(bands)
    output_tif = sys.argv[1] + "imagenes/" + ruta + "/salida/imagen_combinada_" + str(i+1) + ".tif"
    
    with rasterio.open(output_tif, "w", **meta) as dst:
        for j in range(stacked_array.shape[0]):
            dst.write(stacked_array[j], j + 1)
    salidas_raster.append(output_tif)
    print(f"Archivo guardado en: {output_tif}")


Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\GRANULE\L2A_T30TUN_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_1.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\GRANULE\L2A_T30TUP_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_2.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\GRANULE\L2A_T30TVN_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_3.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\GRANULE\L2A_T30TVP_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_4.tif


Ahora quedaría hacer el mosaico. He pensado que se puede hacer lo mismo para las rutas,
o en el bucle anterior una modificacion para que añada la ruta a otra lista. Lo haré en otro momentou

In [6]:
salidas_raster

['E:\\advprog\\p1\\Geodatos\\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\\GRANULE\\L2A_T30TUN_A002395_20250219T111258\\IMG_DATA\\R20m/salida/imagen_combinada_1.tif',
 'E:\\advprog\\p1\\Geodatos\\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\\GRANULE\\L2A_T30TUP_A002395_20250219T111258\\IMG_DATA\\R20m/salida/imagen_combinada_2.tif',
 'E:\\advprog\\p1\\Geodatos\\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\\GRANULE\\L2A_T30TVN_A002395_20250219T111258\\IMG_DATA\\R20m/salida/imagen_combinada_3.tif',
 'E:\\advprog\\p1\\Geodatos\\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\\GRANULE\\L2A_T30TVP_A002395_20250219T111258\\IMG_DATA\\R20m/salida/imagen_combinada_4.tif']

Las rutas se añaden al objeto "salidas raster"

In [9]:
from rasterio.merge import merge

# Abrimos los archivos de salida
src_files_to_mosaic = [rasterio.open(file) for file in salidas_raster]

# juntamos las imagnees con merge
mosaic, out_transform = merge(src_files_to_mosaic, method="first")  # "first" prioriza el primer píxel encontrado

# copiamos los metadatos
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "height": mosaic.shape[1],  
    "width": mosaic.shape[2],   
    "transform": out_transform  
})

# Guardar la imagen final
output_tif = sys.argv[1] + "imagenes\\mosaico.tif"
with rasterio.open(output_tif, "w", **out_meta) as dest:
    for i in range(mosaic.shape[0]):  # (Para mantener el orden de las bandas)
        dest.write(mosaic[i], i + 1)

print(f"Mosaico guardado en: {output_tif}")

# Y cerrar los rasters
for src in src_files_to_mosaic:
    src.close()


Mosaico guardado en: E:\advprog\p1\Geodatos\imagenes\mosaico.tif


In [9]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import mapping

# rutas
output_tif = sys.argv[1] + "imagenes\\mosaico.tif"
shapefile_path = sys.argv[1] + "shp_cantabria\\cantabria.shp"
output_path = sys.argv[1] + "resultados\\recorte_final.tif"


# Cargar shapefile
shapefile = gpd.read_file(shapefile_path)

# Abrir el ráster para obtener su SRC
with rasterio.open(output_tif) as src:
    raster_crs = src.crs  # Obtener CRS del ráster

# Reproyección del shapefile al SRC del ráster si fuera necesario (en este caso lo es)
if shapefile.crs != raster_crs:
    shapefile = shapefile.to_crs(raster_crs)

# Convertir geometría a formato GeoJSON (con shapely)
geometries = [mapping(geom) for geom in shapefile.geometry]

# Abrir el ráster y recortar
with rasterio.open(output_tif) as src:
    out_image, out_transform = mask(src, geometries, crop=True, all_touched=True)

    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # Guardar el nuevo ráster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)

print(f"✅ Recorte guardado en: {output_path}")

✅ Recorte guardado en: E:\advprog\p1\Geodatos\resultados\recorte_final.tif
